In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
from tabGAN import TabGAN
from src import constants as const
import utils
from utils.tictoc import *
from utils.timer import Timer

imports_path = "src//imports.ipynb"
%run "$imports_path"

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [3]:
print(os.path.join(const.dir.data(), "df_adult_edited_train.csv"))

\\sambaad.stud.ntnu.no\arneir\Master-thesis-cf\data\df_adult_edited_train.csv


In [4]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [5]:
n_epochs = 100
n_critic = 10
opt_lr = 0.002
adam_beta1 = 0.5
noise_discrete_unif_max = 0

batch_size = 500
ckpt_every = 1
loss_plot_update_every = 10

timer_path = os.path.join(const.dir.timers(), "timer_tabGAN_speed.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer

In [10]:
@tf.function(experimental_compile=True)
def add(a, b):
    return a+b

add(5,2)

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [14]:
tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=False)

if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start("jit_compile=False")
    tg_qtr.train(n_epochs, batch_size = batch_size, restart_training = True,
             progress_bar=True)
    timer_tabGAN.stop("jit_compile=False")
    timer_tabGAN.save(timer_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time for jit_compile=False: 575.417 seconds


In [15]:
tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = opt_lr, adam_beta1 = adam_beta1,
                quantile_transformation_int = True, quantile_rand_transformation = True,
                noise_discrete_unif_max = noise_discrete_unif_max, jit_compile_train_step=True)

if retrain:
    timer_tabGAN = utils.timer.load(timer_path)
    timer_tabGAN.start("jit_compile=True")
    tg_qtr.train(n_epochs, batch_size = batch_size, restart_training = True,
             progress_bar=True)
    timer_tabGAN.stop("jit_compile=True")
    timer_tabGAN.save(timer_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed time for jit_compile=True: 432.607 seconds


In [18]:
timer_tabGAN = utils.timer.load(timer_path)
timer_tabGAN.elapsed_time["jit_compile=True"] / timer_tabGAN.elapsed_time["jit_compile=False"]

0.7518136797897509